In [1]:
from masters_degree_project.adapters.llm_adapter import LLMAdapter

llm_adapter = LLMAdapter(base_url="http://localhost:8085/v1", api_key="masters_degree_project",
                         model_name="gpt-3.5-turbo")
import os
os.environ['OPENAI_API_KEY'] = "masters_degree_project"
os.environ['OPENAI_API_BASE'] = "http://localhost:8085/v1"
os.environ['USER_AGENT'] = 'myagent'

In [2]:
from tqdm import tqdm
from masters_degree_project.services.behaviour_indexing import behaviour_indexing
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from masters_degree_project.adapters.custom_openai_embeddings import LLAMA_CPP_Compatible_OpenAIEmbeddings as OpenAIEmbeddings
import bs4


# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
# modified_splits = []
# for split in tqdm(splits):
#     modified_splits.append(behaviour_indexing(split.page_content, llm_adapter))

In [3]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [4]:
from masters_degree_project.services.behaviour_indexing import behaviour_indexing

behaviour_indexing(splits[98].page_content, llm_adapter)

"virtual characters' interaction within a simulated setting"

In [5]:
from masters_degree_project.adapters.behaviour_indexing_embeddings import BehaviourIndexingEmbeddings

modified_vectorstore = FAISS.from_documents(documents=splits, embedding=BehaviourIndexingEmbeddings(llm_adapter=llm_adapter, openai_embeddings=OpenAIEmbeddings()))

modified_retriever = modified_vectorstore.as_retriever()

100%|██████████| 170/170 [07:18<00:00,  2.58s/it]


In [6]:

from langchain_openai import ChatOpenAI
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


C:\Users\gmora\Documents\GitHub\masters-degree-project\venv\Lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)
modified_rag_chain = (
        {"context": modified_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

In [15]:
rag_chain.invoke("What is Chain of Hindsight?")

' Chain of Hindsight (CoH) is a technique that improves model outputs by presenting it with past outputs annotated with human feedback. It encourages the model to learn from its previous mistakes and utilize this information for better performance on complex tasks. The method involves using a sequence of past outputs, each labeled with ratings and corresponding explanations (CoT).'

In [16]:
modified_rag_chain.invoke("What is Chain of Hindsight?")

' I\'m sorry, but based on the provided context, there is no information about "Chain of Hindsight." The given text discusses memory types in human brains, generative agent architecture, proof-of-concept examples, and challenges related to LLM agents. Therefore, I don\'t know about Chain of Hindsight from this context.'

In [17]:
rag_chain.invoke("How is it called when an agent breaks down large tasks into smaller tasks?")

' The process of breaking down large tasks into smaller, manageable subtasks is called task decomposition. This approach allows agents to efficiently handle complex tasks by organizing them into simpler components.'

In [18]:
modified_rag_chain.invoke("How is it called when an agent breaks down large tasks into smaller tasks?")

' The process of breaking down large tasks into smaller, manageable subtasks is called "task decomposition." This approach allows agents to handle complex tasks more efficiently by organizing them into a sequence of simpler steps or goals.'

In [19]:
rag_chain.invoke("The LLM is provided with a list of tool names")

' Yes, an LLM (Language Learning Model) can be provided with a list of tool names along with their descriptions and input/output details to enhance its capabilities when answering user prompts using external tools like Plugins or API calls. This approach is exemplified by models such as TALM and Toolformer that are fine-tuned for tool usage.'

In [20]:
modified_rag_chain.invoke("The LLM is provided with a list of tool names")

' The LLM is provided with a list of tool names, their utility descriptions, and input/output details to assist in answering user prompts using the ReAct format. Tools include executing Python files, generating images, sending tweets, performing no action, or completing tasks.'

In [25]:
rag_chain.invoke("develop a novel anticancer drug step by step")

' To develop a novel anticancer drug step by step using ChemCrow, start by researching current trends in anticancer drug discovery. Then select an appropriate target for the drug and request a suitable scaffold that targets these compounds. Finally, attempt to synthesize the identified compound using organic synthesis tools integrated within LangChain workflow.'

In [26]:
modified_rag_chain.invoke("develop a novel anticancer drug step by step")

" To develop a novel anticancer drug step by step, first identify an oncogenic target relevant to cancer progression. Then select a suitable scaffold that can interact with this target effectively. Finally, design and synthesize the compound based on computational models and experimental validation for efficacy against the chosen target.\n\nNote: The provided context does not directly relate to anticancer drug development but rather discusses risks associated with illicit drugs and bioweapons. However, I've tailored a response using relevant steps in drug discovery as per your request."